In [2]:
import pandas as pd
import numpy as np
from requests import get
import json
import re
import ipaddress
import math
import pandas_explode 
pandas_explode.patch()

# I) Get price according to IP

### 1) read csv file

In [232]:
df = pd.read_csv("products.csv", delimiter=";")

In [184]:
df.head()

,username,ip_address,product,price,infos
0,ldrover0,666.666.666.666,Clam - Cherrystone,712.8,May contain sugar
1,kizakov1,nope,Soup - Campbells Bean Medley,379.26,Contains peanut and fish
2,abromet2,240.177.79.234,Island Oasis - Lemonade,305.96,Ingredients: mustard and fish
3,kkarolowski3,26.191.237.49,"Water - Mineral, Natural",350.15,Contains gluten
4,mbuckney4,58.90.204.239,Radish - Pickled,949.79,"May contain sugar, egg and fish"


### 2) define get location and currency information from ip address using https://ipgeolocation.io/documentation/ip-geolocation-api.html

In [185]:
def location(ip):
    API_key = "7e600ecdc3d84178b7b6c1fe86045c64"
    url = "https://api.ipgeolocation.io/ipgeo?apiKey=" + API_key + "&ip=" + ip 
    response = get(url)
    if response.status_code > 400 : # ip non valide
        return(None, None, None)
    else:
        return(response.json()["country_name"], response.json()["currency"]["name"], 
               response.json()["currency"]["code"] )
    
# Vectorize form
location_vect = np.vectorize(location)

### 3) get info from ip in the dataset

In [186]:
country_name, currency_name, currency_code = location_vect(df.ip_address)

### 4) insert values in the dataframe  

In [187]:
df_c = df.copy()
df_c.insert(2, "ip_location", country_name)
df_c.insert(5, "currency_code", currency_code)

In [188]:
df_c.head()

,username,ip_address,ip_location,product,price,currency_code,infos
0,ldrover0,666.666.666.666,None,Clam - Cherrystone,712.8,None,May contain sugar
1,kizakov1,nope,None,Soup - Campbells Bean Medley,379.26,None,Contains peanut and fish
2,abromet2,240.177.79.234,None,Island Oasis - Lemonade,305.96,None,Ingredients: mustard and fish
3,kkarolowski3,26.191.237.49,United States,"Water - Mineral, Natural",350.15,USD,Contains gluten
4,mbuckney4,58.90.204.239,Japan,Radish - Pickled,949.79,JPY,"May contain sugar, egg and fish"


### 5) drop None value where IP address can't be localized

In [189]:
df_c2 = df_c.dropna(subset = ["ip_location"]).copy()
# remove the currency symbol if exists
s = df_c2.price.str.split().str[0]
df_c2.loc[:, ["price"]] = s.values.reshape(s.shape[0],1)

In [190]:
df_c2["price"] = pd.to_numeric(df_c2.price)

### 6) get current currency exchange rates and convert currencies in eur

In [3]:
currencies_rate = get("http://www.floatrates.com/daily/eur.json").json()

In [191]:
currency_rate = get("https://api.exchangeratesapi.io/latest").json()["rates"] # base is EUR

In [192]:
def converter(currency_rate_table, currency):
    if currency == "EUR":
        return(1)
    elif currency not in currency_rate_table.keys():
        return(0)
    else:
        return(float(currency_rate[currency]))

vectorize_converter = np.vectorize(converter)
eur_price = df_c2.price/vectorize_converter(currency_rate, df_c2.currency_code)

In [193]:
if "price_in_euro" not in df_c2.columns:
    df_c2.insert(5, "price_in_euro", np.round(eur_price,2))
else:
    df_c2["price_in_euro"] = np.round(eur_price,2)

In [195]:
df_c2.head()

,username,ip_address,ip_location,product,price,price_in_euro,currency_code,infos
3,kkarolowski3,26.191.237.49,United States,"Water - Mineral, Natural",350.15,313.39,USD,Contains gluten
4,mbuckney4,58.90.204.239,Japan,Radish - Pickled,949.79,7.83,JPY,"May contain sugar, egg and fish"
7,avowdon7,189.169.17.54,Mexico,Dc Hikiage Hira Huba,111.56,5.21,MXN,Contains sugar
8,epridham8,187.129.113.105,Mexico,Dried Figs,88.05,4.11,MXN,"Ingredients: sugar, milk and fish"
9,tkendrew9,22.32.234.215,United States,Pop - Club Soda Can,861.25,770.83,USD,"May contain peanut, sugar, milk and fish"


# II) Get all ingredient in single column

### 1) get all ingredients

In [231]:
ingredients = []
words = ["and","And","may","May","contain","contains","Contain","Contains","Ingredients","Ingredient","ingredients","ingredient"]
for L in df["infos"].replace(",","", regex=True).str.split(" "): 
    for x in L:
        if x not in ingredients and x not in words:
            ingredients.append(x)
ingredients

['sugar',
 'peanut',
 'fish',
 'Ingredients:',
 'mustard',
 'gluten',
 'egg',
 'milk',
 'soja']

In [234]:
def filterwords(liste_ingredients):
    words_to_filter = ["and","And","may","May","contain","contains","Contain",
                     "Contains","Ingredients","Ingredient","ingredients","ingredient"]
    return([x for x in liste_ingredients if x not in words_to_filter])

In [288]:
df_i = df_c2.copy()

df_i["infos"] = df_i["infos"].replace(",","", regex=True).replace(":","", regex=True).str.split(" ")
df_i["infos"] = df_i["infos"].apply(filterwords)
df_i = df_i.explode("infos")
df_dummies = pd.get_dummies(df_i.infos)
df_i = pd.concat([df_i, df_dummies], axis=1)
df_i.drop(columns = 'infos').head()

,username,ip_address,ip_location,product,price,price_in_euro,currency_code,egg,fish,gluten,milk,mustard,peanut,soja,sugar
3,kkarolowski3,26.191.237.49,United States,"Water - Mineral, Natural",350.15,313.39,USD,0,0,1,0,0,0,0,0
4,mbuckney4,58.90.204.239,Japan,Radish - Pickled,949.79,7.83,JPY,0,0,0,0,0,0,0,1
4,mbuckney4,58.90.204.239,Japan,Radish - Pickled,949.79,7.83,JPY,1,0,0,0,0,0,0,0
4,mbuckney4,58.90.204.239,Japan,Radish - Pickled,949.79,7.83,JPY,0,1,0,0,0,0,0,0
7,avowdon7,189.169.17.54,Mexico,Dc Hikiage Hira Huba,111.56,5.21,MXN,0,0,0,0,0,0,0,1
